<a href="https://colab.research.google.com/github/Biratam/BotUan/blob/main/Biel_canto_(1)_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
import numpy as np
import re
from time import sleep
from requests.api import request

### Configuraçoes do bot

In [2]:
data = requests.get('https://www.totalcorner.com/match/today')

In [3]:
df = pd.read_html(data.text)[0]

In [4]:
df.dropna(axis=1,how='all',inplace=True)

In [5]:
df.dropna(axis=0,how='any',inplace=True)

In [6]:
df.drop(columns=['Handicap','Goal Line','Analysis','Time'],inplace=True)

In [7]:
df.rename(columns={'Unnamed: 3':'Cronometro'},inplace=True)

In [8]:
df['Cronometro'] = df['Cronometro'].apply(lambda a: 45 if a == 'Half'else a).astype('int64')

In [9]:
df['Corner'] = df['Corner'].apply(lambda a: list(map(int,a[0].split(' - '))))

In [10]:
df['Dangerous Attack'] = df['Dangerous Attack'].apply(lambda a: re.findall(r'\d{1,} - \d{1,}',a))

In [11]:
df['Shots'] = df['Shots'].apply(lambda a: re.findall(r'\d{1,} - \d{1,}',a))

In [12]:
df['Dangerous Attack'] = df['Dangerous Attack'].apply(lambda a: a if a != [] else np.nan)
df['Corner'] = df['Corner'].apply(lambda a: a if a != [] else np.nan)
df['Shots'] = df['Shots'].apply(lambda a: a if a != [] else np.nan)

In [13]:
df.dropna(axis=0, how = 'any',inplace=True)

In [14]:
df['Shots'] = df['Shots'].apply(lambda a: list(map(int,a[0].split(' - '))))
df['Dangerous Attack'] = df['Dangerous Attack'].apply(lambda a: list(map(int,a[0].split(' - '))))

In [15]:
df.head()

,League,Cronometro,Home,Score,Away,Corner,Dangerous Attack,Shots
2,Portugal Segunda Liga,91,3 Benfica B,0 - 0,Trofense 4,[6],"[66, 35]","[5, 3]"
3,Portugal Segunda Liga,90,SC Covilha,0 - 1,CD Mafra 4,[5],"[83, 40]","[11, 5]"
4,Turkey Super Lig,89,Antalyaspor,1 - 2,Sivasspor 2,[1],"[70, 36]","[16, 7]"
5,Turkey Super Lig,86,3 Fenerbahce,3 - 2,Istanbulspor 3,[5],"[94, 42]","[11, 4]"
6,Sweden Allsvenskan,92,IFK Varnamo,1 - 0,AIK 2,[8],"[59, 46]","[14, 8]"


In [16]:
def enviar_mensagem(msg):
  bot_token = '6012519840:AAEYh3-Dnmemjb4iHjeEtQECqjB8kBWC19M'
  chat_id = -1001989959215
  url = f'https://api.telegram.org/bot6012519840:AAEYh3-Dnmemjb4iHjeEtQECqjB8kBWC19M/sendMessage?chat_id={chat_id}&text={msg}'
  requests.post(url)



In [17]:
mensagens_enviadas = []

In [18]:
for i in range(len(df)):
  j = df.iloc[i]
  tempo = j['Cronometro'] 

  appm_home = round(j['Dangerous Attack'][0]/tempo,1)
  appm_away = round(j['Dangerous Attack'][0]/tempo,0)

  cg_home = j['Corner'][0] + j['Shots'][0]
  cg_away = j['Corner'][0] + j['Shots'][0]

  condicao_home = appm_home >= 0.3 and cg_home >= 2
  condicao_away = appm_away >= 0.3 and cg_away >= 5  

  t1 = 0 < tempo < 45
  t2 = 45 < tempo < 90

  if (condicao_home or condicao_away) and (t1 or t2):
    
     if t1:
       t = 't1'
     else:
      t = 't2'
     

     home =re.sub(r'^\d{1} ','',j['Home'])
     away =re.sub(r'^\d{1}$','',j['Away']) 
     if f'{home} x {away}_{t}' not in mensagens_enviadas:
      msg = (f'''
        
        CANTO-LIMITE
     Liga:{j["League"]}

     Jogo: {home} x {away}
     Placar: {j ["Score"]}
     Min:{tempo}
     Appm: {appm_home} - {appm_away}
     CG: {cg_home} -{cg_away}
     ''')
     enviar_mensagem(msg)
     mensagens_enviadas.append(f'{home} x {away}_{t}')
     sleep(2)